### MAST30034: Applied Data Science Project 1
---
# Preprocessing Part 2: Aggregating Data by MMWR Week
#### Xavier Travers (1178369)

Aggregate all the data by MMWR week (defined [here](https://ndc.services.cdc.gov/wp-content/uploads/MMWR_Week_overview.pdf)).
This means counting trips to and from each of the boroughs per month.
This is done for each of the taxi types.

In [1]:
# imports used throughout this notebook
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
import os
import sys
import re
from itertools import chain

# add homemade helpers
sys.path.insert(1, '../../scripts')
import helpers.aggregation_helpers as ah
import helpers.join_helpers as jh

# path where the data files are stored
DATA_PATH = '../../data'

In [2]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName('MAST30034 XT Project 1')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.sql.repl.eagerEval.enabled', True) 
    .config('spark.sql.parquet.cacheMetadata', 'true')
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/17 02:25:18 WARN Utils: Your hostname, Polaris resolves to a loopback address: 127.0.1.1; using 172.22.167.96 instead (on interface eth0)
22/08/17 02:25:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/17 02:25:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# read in the population data
pop_df = spark.read.parquet(f'{DATA_PATH}/curated/population_by_borough_by_year')
pop_df.limit(5)

week_year,population,borough
2020,495522.0,Staten Island
2021,493494.0,Staten Island
2018,1629055.0,Manhattan
2019,1628706.0,Manhattan
2019,2253858.0,Queens


### 1. Aggregating the TLC dataset

In [5]:
# read in the cleaned yellow dataset
tlc_df = spark.read.parquet(f'{DATA_PATH}/curated/tlc/cleaned/yellow')

# count the raw # of rows and print it out just to check the formatting
f'{tlc_df.count()} ROWS'
tlc_df.limit(5)

year,month,day,week_ending,week_index,timeline,date,passengers,trip_distance,pu_location_id,do_location_id,hours_elapsed,pu_borough,do_borough,week_year,week_month
2018,10,14,2018-10-20,42,1,10/14/2018,1.0,2.37,43,163,0.17916666666666667,Manhattan,Manhattan,2018,10
2018,10,5,2018-10-06,40,1,10/05/2018,1.0,1.12,230,246,0.1625,Manhattan,Manhattan,2018,10
2018,11,3,2018-11-03,44,1,11/03/2018,1.0,1.0,170,48,0.1597222222222222,Manhattan,Manhattan,2018,10
2018,10,3,2018-10-06,40,1,10/03/2018,1.0,1.7,239,163,0.21666666666666667,Manhattan,Manhattan,2018,10
2018,10,14,2018-10-20,42,1,10/14/2018,1.0,1.64,163,239,0.1897222222222222,Manhattan,Manhattan,2018,10


In [6]:
TLC_COMMON_GROUP_COLUMNS = [
    'week_year',
    'week_month',
    'week_ending',
    'week_index',
    'timeline'
]

TLC_GROUP_BY_PU_COLUMNS = TLC_COMMON_GROUP_COLUMNS + ['pu_borough'];
TLC_GROUP_BY_DO_COLUMNS = TLC_COMMON_GROUP_COLUMNS + ['do_borough'];

# TODO: commenting
TLC_AGGREGATE_COLUMNS = {
    '*': [
        'count', 
        'count_per_capita',
        'count_per_100k'
    ],
    'passengers': [
        # 'total', 
        # 'total_per_capita', 
        # 'daily_average', 
        'average'
    ],
    'trip_distance': [
        # 'total', 
        # 'total_per_capita', 
        # 'daily_average', 
        # 'daily_average_per_capita', 
        'average'
    ],
    # 'hours_elapsed': [
    #     'total', 
    #     'total_per_capita', 
    #     'daily_average', 
    #     'daily_average_per_capita', 
    #     'average'
    # ],
}

#### Group by pick-up location

In [7]:
tlc_by_pu_df = ah.group_and_aggregate(tlc_df, pop_df, TLC_GROUP_BY_PU_COLUMNS, 
    TLC_AGGREGATE_COLUMNS)
# TODO: commenting
# force this into memory 
# otherwise writing parquets results in a java executor out of memory error
tlc_by_pu_df = spark.createDataFrame(tlc_by_pu_df.collect())

In [8]:
tlc_by_pu_df.sort('week_index').limit(5)
# TODO: commenting

week_year,week_month,week_ending,week_index,timeline,pu_borough,population,num_trips,num_pc_trips,num_p100k_trips,avg_passengers,avg_trip_distance
2018,7,2018-07-07,27,1,Brooklyn,2578074.0,20276,0.007864785882794675,786.4785882794675,1.6232491615703295,3.766265042414675
2018,7,2018-07-07,27,1,Manhattan,1629055.0,1393501,0.8554045136597598,85540.45136597598,1.634864991126666,2.301717759800682
2018,7,2018-07-07,27,1,Staten Island,476260.0,16,3.359509511611304...,3.359509511611305,1.6875,7.320625
2018,7,2018-07-07,27,1,Queens,2274605.0,96780,0.042548046803730755,4254.804680373076,1.6782909692085142,10.768574705517555
2018,7,2018-07-07,27,1,Bronx,1432087.0,2029,0.001416813363992...,141.68133639925506,1.6407097092163627,5.085219319862009


In [9]:
tlc_by_pu_df.sort('avg_trip_distance', ascending=False).limit(5)

week_year,week_month,week_ending,week_index,timeline,pu_borough,population,num_trips,num_pc_trips,num_p100k_trips,avg_passengers,avg_trip_distance
2021,3,2021-03-13,167,2,Staten Island,493494.0,51,1.033447215163710...,10.334472151637103,1.0784313725490196,32.103529411764704
2020,9,2020-09-19,142,2,Staten Island,495522.0,39,7.870488091346096E-5,7.870488091346096,1.0,31.242307692307698
2020,12,2021-01-02,157,2,Staten Island,495522.0,48,9.686754573964425E-5,9.686754573964425,1.0,31.226875000000007
2021,1,2021-01-16,159,2,Staten Island,493494.0,59,1.195556582248213...,11.955565822482138,1.0169491525423728,30.70338983050847
2021,6,2021-06-26,182,2,Staten Island,493494.0,42,8.510741771936437E-5,8.510741771936438,1.2619047619047619,30.672142857142862


In [10]:
# filter for only the considered timelines
tlc_by_pu_df = tlc_by_pu_df.where(F.col('timeline') > 0)

In [11]:
tlc_by_pu_df.write.mode('overwrite').parquet(f'{DATA_PATH}/curated/tlc/aggregated/yellow/by_pu')
# TODO: commenting

#### Group by drop-off location

In [12]:
tlc_by_do_df = ah.group_and_aggregate(tlc_df, pop_df, TLC_GROUP_BY_DO_COLUMNS, 
    TLC_AGGREGATE_COLUMNS)
# TODO: commenting
# force this into memory 
# otherwise writing parquets results in a java executor out of memory error
tlc_by_do_df = spark.createDataFrame(tlc_by_do_df.collect())

In [13]:
tlc_by_do_df.sort('week_index').limit(5)
# TODO: commenting

week_year,week_month,week_ending,week_index,timeline,do_borough,population,num_trips,num_pc_trips,num_p100k_trips,avg_passengers,avg_trip_distance
2018,7,2018-07-07,27,1,Brooklyn,2578074.0,65629,0.02545660054754053,2545.660054754053,1.6292187904737234,7.081771320605249
2018,7,2018-07-07,27,1,Manhattan,1629055.0,1357406,0.8332474962478247,83324.74962478246,1.6366584500142183,2.300199174012804
2018,7,2018-07-07,27,1,Staten Island,476260.0,316,6.635031285432327E-4,66.35031285432326,1.639240506329114,21.03791139240506
2018,7,2018-07-07,27,1,Queens,2274605.0,77259,0.03396589737558829,3396.589737558829,1.6567778511241409,8.181016451157799
2018,7,2018-07-07,27,1,Bronx,1432087.0,11992,0.00837379293297125,837.3792932971251,1.6533522348232155,9.226953802534991


In [14]:
tlc_by_do_df.sort('avg_trip_distance', ascending=False).limit(5)

week_year,week_month,week_ending,week_index,timeline,do_borough,population,num_trips,num_pc_trips,num_p100k_trips,avg_passengers,avg_trip_distance
2021,3,2021-03-13,167,2,Staten Island,493494.0,124,2.512695189809805E-4,25.126951898098053,1.2580645161290323,24.661129032258074
2020,12,2020-12-05,153,2,Staten Island,495522.0,114,2.300604211316551E-4,23.00604211316551,1.2456140350877194,24.417982456140344
2020,12,2021-01-02,157,2,Staten Island,495522.0,119,2.401507904795347E-4,24.01507904795347,1.218487394957983,24.379663865546224
2021,6,2021-06-19,181,2,Staten Island,493494.0,169,3.424560379660138E-4,34.24560379660138,1.5621301775147929,24.308165680473376
2021,2,2021-02-20,164,2,Staten Island,493494.0,123,2.492431518924242E-4,24.92431518924242,1.3333333333333333,24.152195121951227


In [15]:
tlc_by_do_df.write.mode('overwrite').parquet(f'{DATA_PATH}/curated/tlc/aggregated/yellow/by_do')
# TODO: commenting

### 2. Aggregating the COVID dataset

In [16]:
# read in the covid dataset
covid_df = spark.read.parquet(f'{DATA_PATH}/curated/virals/covid/cleaned/cases_by_day')
covid_df.limit(5)
# TODO: commenting

date,week_ending,week_year,week_month,week_index,timeline,cases,deaths,hospitalised,borough
02/29/2020,2020-02-29,2020,2,113,1,0,0,1,Brooklyn
03/01/2020,2020-03-07,2020,3,114,1,0,0,0,Brooklyn
03/02/2020,2020-03-07,2020,3,114,1,0,0,2,Brooklyn
03/03/2020,2020-03-07,2020,3,114,1,0,0,3,Brooklyn
03/04/2020,2020-03-07,2020,3,114,1,1,0,1,Brooklyn


In [17]:
COVID_GROUP_COLUMNS = [
    'week_year',
    'week_month',
    'week_ending',
    'week_index',
    'timeline',
    'borough'
]
# TODO: commenting
COVID_AGGREGATE_COLUMNS = {
    'cases': [
        'total', 
        'total_per_capita', 
        'total_per_100k',
        # 'daily_average', 
        # 'daily_average_per_capita'
    ],
}

In [18]:
covid_df = ah.group_and_aggregate(covid_df, pop_df, COVID_GROUP_COLUMNS, 
    COVID_AGGREGATE_COLUMNS)

# force this into memory 
# otherwise writing parquets results in a java executor out of memory error
covid_df = spark.createDataFrame(covid_df.collect())
# TODO: commenting

In [19]:
covid_df.sort('week_index').limit(5)
# TODO: commenting

week_year,week_month,week_ending,week_index,timeline,borough,population,tot_cases,tot_pc_cases,tot_p100k_cases
2020,2,2020-02-29,113,1,Bronx,1466438.0,0.0,0.0,0.0
2020,2,2020-02-29,113,1,Staten Island,495522.0,0.0,0.0,0.0
2020,2,2020-02-29,113,1,Brooklyn,2727393.0,0.0,0.0,0.0
2020,2,2020-02-29,113,1,Queens,2395791.0,0.0,0.0,0.0
2020,2,2020-02-29,113,1,Manhattan,1687834.0,1.0,5.924753263650335E-7,0.059247532636503356


In [20]:
# save it
# TODO: commenting
covid_df.write.mode('overwrite').parquet(f'{DATA_PATH}/curated/virals/covid/aggregated/cases_by_week')

### 3. Aggregating the Flu dataset
*The flu dataset is already grouped by MMWR week, so only daily_averages can be calculated*

In [21]:
# read in the flu dataset
flu_df = spark.read.parquet(f'{DATA_PATH}/curated/virals/flu/cleaned/cases_by_week')
flu_df.limit(5)

year,month,day,week_ending,week_year,week_month,week_index,timeline,date,borough,disease,cases
2019,2,16,2019-02-16,2019,2,59,1,02/16/2019,Staten Island,INFLUENZA_UNSPECI...,0
2018,12,22,2018-12-22,2018,12,51,1,12/22/2018,Staten Island,INFLUENZA_B,2
2018,10,13,2018-10-13,2018,10,41,1,10/13/2018,Manhattan,INFLUENZA_B,0
2018,11,3,2018-11-03,2018,10,44,1,11/03/2018,Staten Island,INFLUENZA_A,0
2020,12,12,2020-12-12,2020,12,154,2,12/12/2020,Staten Island,INFLUENZA_B,3


In [22]:
FLU_GROUP_COLUMNS = [
    'week_year',
    'week_month',
    'week_ending',
    'week_index',
    'timeline',
    'borough'
]
# TODO: commenting
FLU_AGGREGATE_COLUMNS = {
    'cases': [
        'total', 
        'total_per_capita', 
        'total_per_100k',
    ],
}

In [23]:
flu_df = ah.group_and_aggregate(flu_df, pop_df, FLU_GROUP_COLUMNS, 
    FLU_AGGREGATE_COLUMNS)

# force this into memory 
# otherwise writing parquets results in a java executor out of memory error
flu_df = spark.createDataFrame(flu_df.collect())
# TODO: commenting

In [24]:
flu_df.sort('week_index').limit(5)
# TODO: commenting

week_year,week_month,week_ending,week_index,timeline,borough,population,tot_cases,tot_pc_cases,tot_p100k_cases
2018,10,2018-10-06,40,1,Brooklyn,2578074.0,5.0,1.939432304891171E-6,0.19394323048911707
2018,10,2018-10-06,40,1,Bronx,1432087.0,10.0,6.982815988134799E-6,0.69828159881348
2018,10,2018-10-06,40,1,Manhattan,1629055.0,6.0,3.683116899061112E-6,0.36831168990611124
2018,10,2018-10-06,40,1,Staten Island,476260.0,0.0,0.0,0.0
2018,10,2018-10-06,40,1,Queens,2274605.0,7.0,3.077457404692243...,0.3077457404692243


In [25]:
flu_df.write.mode('overwrite').parquet(f'{DATA_PATH}/curated/virals/flu/aggregated/cases_by_week')